In [2]:
import numpy as np
import numba

In [3]:
def iou(a: np.ndarray, b: np.ndarray) -> float:
    a_tl, a_br = a[:4].reshape((2, 2))
    b_tl, b_br = b[:4].reshape((2, 2))
    int_tl = np.maximum(a_tl, b_tl)
    int_br = np.minimum(a_br, b_br)
    int_area = np.product(np.maximum(0., int_br - int_tl))
    a_area = np.product(a_br - a_tl)
    b_area = np.product(b_br - b_tl)
    return int_area / (a_area + b_area - int_area)

In [4]:
def calc_iou_matrix(detections,trackers):
    iou_matrix = np.zeros((len(detections), len(trackers)), dtype=np.float32)

    for d, det in enumerate(detections):
        for t, trk in enumerate(trackers):
            iou_matrix[d, t] = iou(det, trk)
    return iou_matrix

In [5]:
A = np.random.rand(100,4)
B = np.random.rand(100,4)

In [5]:
%timeit calc_iou_matrix(A,B)

328 ms ± 26.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
@numba.jit(nopython=True)
def product(a):
    result = 1
    for i in range(len(a)):
        result*=a[i]
    return result

@numba.jit(nopython=True)
def iou(a: np.ndarray, b: np.ndarray) -> float:
    a_tl, a_br = a[0:2],a[2:4]
    b_tl, b_br = b[0:2],b[2:4]
    int_tl = np.maximum(a_tl, b_tl)
    int_br = np.minimum(a_br, b_br)
    int_area = product(np.maximum(0., int_br - int_tl))
    a_area = product(a_br - a_tl)
    b_area = product(b_br - b_tl)
    return int_area / (a_area + b_area - int_area)
%timeit calc_iou_matrix(A,B)

14.5 ms ± 792 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
@numba.jit(nopython=True)
def calc_iou_matrix(detections,trackers):
    iou_matrix = np.zeros((len(detections), len(trackers)), dtype=np.float32)

    for d in range(len(detections)):
        det = detections[d]
        for t in range(len(trackers)):
            trk = trackers[t]
            iou_matrix[d, t] = iou(det, trk)

%timeit calc_iou_matrix(A,B)

7.08 ms ± 31 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
@numba.vectorize
def custom_operation(a,b):
    if b == 0:
        return 0
    return a*b if a>b else a/b
custom_operation(A,B)
custom_operation(A,np.ones((1,4)))

array([[2.84786978e-01, 1.49667930e-01, 4.52272718e-01, 7.64122472e-01],
       [1.29328195e-01, 1.85575401e-01, 2.44202698e-01, 1.62779569e-01],
       [8.79994339e-01, 7.84159368e-01, 8.96072205e-01, 4.64633477e-01],
       [2.34234905e-01, 8.16629721e-02, 9.31725878e-01, 9.15272896e-02],
       [6.17814797e-01, 7.92031790e-01, 5.93997225e-01, 9.17106773e-01],
       [5.46257280e-01, 5.90488641e-01, 2.45114229e-01, 5.27682917e-02],
       [7.13898454e-01, 8.35825322e-01, 6.37713557e-01, 3.53566190e-01],
       [7.24266392e-01, 8.09674239e-01, 6.40267349e-01, 2.10335049e-02],
       [3.60260021e-01, 6.98047814e-01, 7.49194667e-01, 7.24404182e-01],
       [4.54322591e-01, 9.39216612e-01, 1.76701033e-01, 5.49275464e-01],
       [8.48035985e-01, 5.33716147e-01, 9.57733994e-01, 9.79705353e-01],
       [9.78277570e-01, 4.44890125e-01, 9.39733444e-01, 7.50692101e-02],
       [8.22548577e-01, 6.41685401e-01, 7.45234802e-01, 5.09103658e-01],
       [2.60426971e-01, 1.00493773e-01, 1.71217758e

In [18]:
@numba.guvectorize(['(f8[:, :], f8[:, :], f8[:, :])'], '(m,k),(n,k1)->(m, n)')
def calc_iou_matrix(x, y, z):
    for i in range(x.shape[0]):
        for j in range(y.shape[1]):
            z[i, j] = iou(x[i],y[i])

%timeit calc_iou_matrix(A,B)

196 µs ± 2.46 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [9]:
from numba import cuda

In [10]:
@cuda.jit(device=True)
def iou(a: np.ndarray, b: np.ndarray) -> float:
    xx1 = max(a[0], b[0])
    yy1 = max(a[1], b[1])
    xx2 = min(a[2], b[2])
    yy2 = min(a[3], b[3])
    w = max(0., xx2 - xx1)
    h = max(0., yy2 - yy1)
    wh = w * h
    result = wh / ((a[2]-a[0])*(a[3]-a[1])
        + (b[2]-b[0])*(b[3]-b[1]) - wh)
    return result

In [ ]:
@numba.guvectorize(['(f8[:, :], f8[:, :], f8[:, :])'], '(m,k),(n,k1)->(m, n)',target="cuda")
def calc_iou_matrix(x, y, z):
    for i in range(x.shape[0]):
        for j in range(y.shape[1]):
            z[i, j] = iou(x[i],y[j])